In [ ]:
import numpy as np #importa a biblioteca usada para trabalhar com vetores de matrizes
%run normalization_utils.py

X, Y, feature_names = build_bow()

In [ ]:
# Testando metodo para converter uma string numa amostra para predição
sentence = 'Tinha uma pedra no meio do caminho, no meio do caminho tinha uma pedra. Qual é o sentido da vida se a nossa existência é tão efemera.'

a = sentence_to_features(sentence, feature_names)

print(a[a!=0])
print(len(a))

In [ ]:
print('Calculando folds...')
folds = stratified_kfolds(Y, 5)
print('Folds calculados')

In [ ]:
Xtrain = X[folds[0][0]]
Ytrain = Y[folds[0][0]]

Xtest = X[folds[0][1]]
Ytest = Y[folds[0][1]]

In [ ]:
%run Methods/Kneighbors.py
%run accuracy.py

bestAccuracy = 0

for i in range(1, 101, 2):
    Ypred = testSamples(Xtrain, Ytrain, Xtest, i)
    acc = accuracy(Ypred, Ytest)
    print('K: ', i, ' acc: ', acc)
    if bestAccuracy < acc:
        bestAccuracy = acc
        k = i
print('Best -> K: ', k, ' acc: ', bestAccuracy)

In [ ]:
%run Methods/logisticRegression.py
%run accuracy.py

lambda_reg = 100

theta, Ypred = regression(Xtrain, Ytrain, Xtest, lambda_reg)
display(accuracy(Ypred, Ytest))

In [ ]:
%run Methods/naiveBayes.py
%run accuracy.py

Ypred = naiveBayes(Xtrain, Ytrain, Xtest)
display(accuracy(Ypred, Ytest))

In [ ]:
%run Methods/neuralNetworks.py
%run accuracy.py

vLambda = 10

Ypred, Theta1, Theta2 = neuralNetwork(Xtrain, Ytrain, Xtest, vLambda)
display(accuracy(Ypred, Ytest))

In [ ]:
%run Methods/svmMethod.py
%run accuracy.py

kernel = 0
cost = 4
gamma = 1

model, Ypred = svmUse(Xtrain, Ytrain, Xtest, Ytest, kernel, cost, gamma)
acc = accuracy(Ypred, Ytest)
print('cost:', cost, ' kernel:', kernel, ' gamma:', gamma, 'accuracy:', acc)